In [1]:
import pandas as pd

# Parameters
pv_file = 'data\\ninja_pv_country_GB_merra-2_corrected (1).csv'
wind_file = 'data\\ninja_wind_country_GB_current-merra-2_corrected.csv'
output_file = 'processed_pv_wind.csv'

# Read PV data, skipping header rows
pv_df = pd.read_csv(pv_file, skiprows=2)

# Read Wind data, selecting only time and onshore columns
wind_df = pd.read_csv(wind_file, skiprows=2, usecols=['time', 'offshore'])

# Convert time to datetime, explicitly setting UTC timezone
pv_df['time'] = pd.to_datetime(pv_df['time'], utc=True)
wind_df['time'] = pd.to_datetime(wind_df['time'], utc=True)

# Extract year
pv_df['year'] = pv_df['time'].dt.year
wind_df['year'] = wind_df['time'].dt.year

# Calculate hour of the year (hours since start of year)
pv_df['hour_of_year'] = (pv_df['time'] - pd.to_datetime(pv_df['year'].astype(str) + '-01-01', utc=True)).dt.total_seconds() / 3600
wind_df['hour_of_year'] = (wind_df['time'] - pd.to_datetime(wind_df['year'].astype(str) + '-01-01', utc=True)).dt.total_seconds() / 3600

# Remove last day of leap years (hours >= 8760)
def filter_leap_year(df_year):
    year = df_year['year'].iloc[0]
    is_leap = (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0)
    if is_leap:
        return df_year[df_year['hour_of_year'] < 8760]
    return df_year

pv_df = pv_df.groupby('year').apply(filter_leap_year).reset_index(drop=True)
wind_df = wind_df.groupby('year').apply(filter_leap_year).reset_index(drop=True)

# Rename columns
pv_df = pv_df[['year', 'hour_of_year', 'national']].rename(columns={'national': 'PV'})
wind_df = wind_df[['year', 'hour_of_year', 'offshore']].rename(columns={'offshore': 'Wind'})

# Merge PV and Wind data on year and hour_of_year
merged_df = pd.merge(pv_df, wind_df, on=['year', 'hour_of_year'], how='inner')

# Handle missing or invalid data
merged_df = merged_df.dropna()
merged_df = merged_df[
    merged_df['PV'].notnull() & merged_df['PV'].apply(lambda x: isinstance(x, (int, float))) &
    merged_df['Wind'].notnull() & merged_df['Wind'].apply(lambda x: isinstance(x, (int, float)))
]

merged_df["hour_of_year"] = merged_df["hour_of_year"]+1

# Save to CSV
merged_df.to_csv(output_file, index=False)

# Print the first few rows for verification
print(merged_df.head().to_csv(index=False))

C:\Users\user\AppData\Local\Temp\ipykernel_4224\1809242341.py:34: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  pv_df = pv_df.groupby('year').apply(filter_leap_year).reset_index(drop=True)
C:\Users\user\AppData\Local\Temp\ipykernel_4224\1809242341.py:35: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  wind_df = wind_df.groupby('year').apply(filter_leap_year).reset_index(drop=True)


year,hour_of_year,PV,Wind
1980,1.0,0.0,0.2858
1980,2.0,0.0,0.2827
1980,3.0,0.0,0.2852
1980,4.0,0.0,0.292
1980,5.0,0.0,0.3039



In [2]:
years = merged_df['year'].unique()[-40:]
filtered_df = merged_df[merged_df['year'].isin(years)]
filtered_df.to_csv('data_VRE_40offshore.csv', index=False)